In [1]:
#reading the soundfile

In [2]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav


In [3]:
(rate,sig) = wav.read("/home/shivam/Desktop/python codes/LDC2007S10.wav")
data = mfcc(sig,rate)

In [4]:
print(data.shape)

(179999, 13)


In [6]:
print (data)
data=data.reshape(data.shape[0],13)
data.shape

[[ 19.6884738    0.34520428 -18.96777002 ...  17.19743708   7.23626649
  -30.71404477]
 [ 19.6776887    6.5152444  -18.13150971 ...  36.56473388  15.91031454
  -20.585949  ]
 [ 19.38906465   4.09170589 -16.16741236 ...  35.76400799   9.97291938
  -27.12420088]
 ...
 [  4.48181023 -28.49382144  -1.39828052 ...   6.00959207   4.32710969
  -11.78033898]
 [  4.59760795 -31.19398402  -1.50272759 ...   0.10406072  -5.33376854
  -17.42454733]
 [  4.3957976  -30.3591145    0.93997567 ...   1.45635257  -3.36968642
  -10.39677659]]


(179999, 13)

In [7]:
print(data[0])

[ 19.6884738    0.34520428 -18.96777002 -18.91139043 -28.31415639
  -5.61446044 -28.95105212 -34.86181481  11.5893669   -4.58225194
  17.19743708   7.23626649 -30.71404477]


In [8]:
import soundfile as sf
from sklearn import cluster
import numpy as np
import pandas as pd
dataset, fs = sf.read('/home/shivam/Desktop/python codes/LDC2007S10.wav')

In [9]:
print [dataset]

[array([1.29913330e-01, 9.65881348e-02, 5.27648926e-02, ...,
       0.00000000e+00, 3.05175781e-05, 3.05175781e-05])]


In [9]:
#reshape the data 

In [10]:
dataset=dataset.reshape(dataset.shape[0],1)
ks=range(1,2)
dataset.shape

(28800000, 1)

In [11]:
Y=dataset[0:100]
Y.shape

(100, 1)

In [12]:
KMeans = [cluster.KMeans(n_clusters = i, init="k-means++").fit(data[179989:179999]) for i in ks]

In [13]:
print(len(KMeans))

1


In [14]:
#code for calculating Bayesian Information Criteria for a cluster of data

In [15]:
import numpy as np
from scipy.spatial import distance
def compute_bic(kmeans,X):
    """
    Computes the BIC metric for a given clusters

    Parameters:
    -----------------------------------------
    kmeans:  List of clustering object from scikit learn

    X     :  multidimension np array of data points

    Returns:
    -----------------------------------------
    BIC value
    """
    # assign centers and labels
    centers = [kmeans.cluster_centers_]
    labels  = kmeans.labels_
    #number of clusters
    
    m = kmeans.n_clusters
    #print(m)
    #print m
    #print (m)
    # size of the clusters
    n = np.bincount(labels)
    #size of data set
    N, d = X.shape
    #print m,N
    #print(X.shape)
    #print(d)
    #tmp=(1.0 / (N - m) / d) *sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 'euclidean')**2) for i in range(m)])
    #print(tmp)
    #compute variance for all clusters beforehand
    #print(N-m)
    cl_var = (1.0 / (N - m) / d) * sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 'euclidean')**2) for i in range(m)])

    const_term = 0.5 * m * np.log(N) * (d+1)

    BIC = np.sum([n[i] * np.log(n[i]) -
               n[i] * np.log(N) -
             ((n[i] * d) / 2) * np.log(2*np.pi*cl_var) -
              ((n[i] - 1) * d/ 2) for i in range(m)]) - const_term

    return(BIC)

In [16]:
print(KMeans[0])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=1, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [17]:
#compute BIC

In [18]:
BIC = [compute_bic(kmeansi,data[179989:179999]) for kmeansi in KMeans]
#BIC = compute_bic(kmeansi,dataset) 
print(BIC)

[-342.4814753693512]


In [22]:
 #code for detecting Speaker change  

In [86]:
#!/usr/bin/python
a=0
t=a+200
b=a+400
threshold= 480 #flexible as per penalty added and our knowledge too
count=0
for i in range(200,len(data)): 
    #print a,t,b
    X1= data[a:t]
    X2= data[t:b]
    X=  data[a:b]
    #print("helllo")
    if(X2.shape[0]<200):
        break        
    #compute_bic(X1) compute BIC(X2)
    bic1= BIC = [compute_bic(kmeansi,X1) for kmeansi in KMeans]
    bic2= BIC = [compute_bic(kmeansi,X2) for kmeansi in KMeans]
    bic= BIC = [compute_bic(kmeansi,X) for kmeansi in KMeans]
    diff= abs((bic1[0]+bic2[0])-bic[0])
    #print bic1[0]
    #print bic2[0]
    
    if diff>threshold:
                print "speaker change detected at ",t
                count=count+1
                a=b
                t=a+200
                b=a+400
                #print a,t,b
    else:
                t=t+1
                a=t-200
                b=t+200
        

speaker change detected at  474
speaker change detected at  938
speaker change detected at  1481
speaker change detected at  1918
speaker change detected at  2318
speaker change detected at  4052
speaker change detected at  4682
speaker change detected at  5234
speaker change detected at  5850
speaker change detected at  6374
speaker change detected at  6910
speaker change detected at  7525
speaker change detected at  7929
speaker change detected at  8594
speaker change detected at  9197
speaker change detected at  9624
speaker change detected at  10062
speaker change detected at  10526
speaker change detected at  14949
speaker change detected at  15870
speaker change detected at  20414
speaker change detected at  21151
speaker change detected at  21831
speaker change detected at  22332
speaker change detected at  22748
speaker change detected at  23478
speaker change detected at  24061
speaker change detected at  24614
speaker change detected at  25032
speaker change detected at  2568

In [87]:
print (count)

184
